In [1]:
import pandas as pd
import numpy as np

import os
path="../../Data/"
import numpy as np
import re
import datetime as dt

In [2]:
def onehotencoder(df,cols,excwords,maxrecs=10,mincount=0.8):
    
    #create nulls where unknown for future imputation
    for col in df.columns:
        mask_exc=(df[col].isin(excwords))
        df[col][mask_exc]=np.nan  
    ohe_cols=\
    [col for col in cols if len(df[col].value_counts())<maxrecs
     and df[col].count()/df[col].shape[0]>mincount]
    
    print(r'Total ohe variables = %0.0f ' % (len(ohe_cols)))
        
    df_ohe_cols=df[ohe_cols]
         
    df_out=pd.get_dummies(df_ohe_cols, prefix=df_ohe_cols.columns,
               columns=df_ohe_cols.columns)
       
    df_out['eid']=df['eid'].tolist()
    
    return df_out

In [3]:
def findcols(df,string):
    return [col for col in df if string in col]

In [4]:
df_model=pd.read_parquet(path+'df_model.parquet')

In [5]:
apoe4_df=pd.read_parquet(path+'genotype.parquet')

In [6]:
apoe4_df=apoe4_df[pd.notnull(apoe4_df['Genotype'])]

In [7]:
apoe4_df=onehotencoder(apoe4_df,['Genotype'],[],maxrecs=10,mincount=0.7)

Total ohe variables = 1 


In [8]:
dis_ohe=pd.read_parquet(path+'dis_ohe.parquet')
#dis_ohe_icd10=pd.read_parquet(path+'dis_ohe_icd10.parquet')
dis_ohe_icd10=pd.read_parquet(path+'dis_ohe_icd10_20210924.parquet')
labels=pd.read_parquet(path+'labels.parquet')
excludes=pd.read_parquet(path+'excludes.parquet')

In [9]:
#dis_ohe['total_dis']=dis_ohe[[col for col in dis_ohe.columns if 'eid' not in col]].sum(axis=1)

In [10]:
findcols(dis_ohe,'total_dis')

['total_dis']

In [11]:
ukb_deaths=pd.read_parquet(path+'ukb_tp0_new.parquet',columns=['eid','date_of_death_f40000_0_0'])
ukb_deaths=ukb_deaths[(ukb_deaths['date_of_death_f40000_0_0']!='nan')]
ukb_deaths['date_of_death_f40000_0_0']=pd.to_datetime(ukb_deaths['date_of_death_f40000_0_0'])

In [12]:
mask_dem=~(labels[[col for col in labels.columns if 'dementia' in col]].sum(axis=1)>0)
mask_pd=~(labels[[col for col in labels.columns if 'parkinso' in col]].sum(axis=1)>0)
mask_ad=~(labels[[col for col in labels.columns if 'alzhei' in col]].sum(axis=1)>0)



'''
Exclude everyone who died from something other than dementia
'''
dem_excs=list(excludes[(excludes[[col for col in excludes.columns if 'dementia' in col]].sum(axis=1)>0)]['eid'])
death_exc_dem=list(pd.merge(ukb_deaths,labels[mask_dem],on='eid',how='inner')['eid'])
eids_exc_dem=list(dem_excs)+list(death_exc_dem)

PD_excs=list(excludes[(excludes[[col for col in excludes.columns if 'parkinson' in col]].sum(axis=1)>0)]['eid'])
death_exc_pd=list(pd.merge(ukb_deaths,labels[mask_pd],on='eid',how='inner')['eid'])
eids_exc_pd=list(PD_excs)+list(death_exc_pd)


AD_excs=list(excludes[(excludes[[col for col in excludes.columns if 'AD' in col]]\
.sum(axis=1)>0)]['eid'])+dem_excs
death_exc_ad=list(pd.merge(ukb_deaths,labels[mask_ad],on='eid',how='inner')['eid'])
eids_exc_ad=list(AD_excs)+list(death_exc_ad)



In [13]:
dis_ohe_icd10['eid']=dis_ohe_icd10['eid'].astype(str)
df_model['eid']=df_model['eid'].astype(str)
dis_ohe['eid']=dis_ohe['eid'].astype(str)
apoe4_df['eid']=apoe4_df['eid'].astype(str)
labels['eid']=labels['eid'].astype(str)

In [14]:
df=pd.merge(df_model,dis_ohe_icd10,on='eid',how='inner')
#df=pd.merge(df,dis_ohe,on='eid',how='inner')
df=pd.merge(df,apoe4_df,on='eid',how='left')

In [15]:
df2=pd.merge(df_model,dis_ohe,on='eid',how='inner')
#df=pd.merge(df,dis_ohe,on='eid',how='inner')
df2=pd.merge(df2,apoe4_df,on='eid',how='left')

In [16]:
findcols(df2,'total_dis')

['total_dis']

In [17]:
findcols(df2,'snap')

['avg_duration_to_first_press_of_snapbutton_in_each_round',
 'total_snap_presses',
 'mean_snap_presses']

In [18]:
labels['dementia']=labels[[col for col in labels.columns if 'dementia' in col]].max(axis=1)
labels['PD']=labels[[col for col in labels.columns if 'parkinson' in col]].max(axis=1)
labels['AD']=labels[[col for col in labels.columns if 'alzh' in col]].max(axis=1)

In [19]:
df=pd.merge(df,labels[['eid','dementia','PD','AD']],on='eid',how='inner')

In [20]:
df2=pd.merge(df2,labels[['eid','dementia','PD','AD']],on='eid',how='inner')

In [21]:
df['AST_ALT_ratio']=df['aspartate_aminotransferase_f30650_0_0']/\
df['alanine_aminotransferase_f30620_0_0']

In [22]:
df2['AST_ALT_ratio']=df2['aspartate_aminotransferase_f30650_0_0']/\
df2['alanine_aminotransferase_f30620_0_0']

In [23]:
'''
Exclude deaths with no disease and already have disease
'''
mask=(df['eid'].isin(eids_exc_dem))
df_dem=df[~mask]
df_dem.drop(columns=[col for col in findcols(df_dem,'dementia|AD') if col!='dementia'],inplace=True)

mask=(df['eid'].isin(eids_exc_pd))
df_pd=df[~mask]
df_pd.drop(columns=[col for col in findcols(df_dem,'parkins') if col!='PD'],inplace=True)

mask=(df['eid'].isin(eids_exc_ad))
df_ad=df[~mask]
df_ad.drop(columns=[col for col in findcols(df_dem,'dement') if col!='AD'],inplace=True)




In [24]:
'''
Exclude deaths with no disease and already have disease
'''
mask=(df2['eid'].isin(eids_exc_dem))
df_dem2=df2[~mask]
df_dem2.drop(columns=[col for col in findcols(df_dem2,'dementia|AD') if col!='dementia'],inplace=True)

mask=(df2['eid'].isin(eids_exc_pd))
df_pd2=df2[~mask]
df_pd2.drop(columns=[col for col in findcols(df_dem2,'parkins') if col!='PD'],inplace=True)

mask=(df2['eid'].isin(eids_exc_ad))
df_ad2=df2[~mask]
df_ad2.drop(columns=[col for col in findcols(df_ad2,'dement') if col!='AD'],inplace=True)



In [25]:
df_dem2.to_parquet(path+'df_dem2_20210924.parquet')
df_pd2.to_parquet(path+'df_pd2_20210924.parquet')
df_ad2.to_parquet(path+'df_ad2_20210924.parquet')

In [26]:
df_dem.to_parquet(path+'df_dem_20210924.parquet')
df_pd.to_parquet(path+'df_pd_20210924.parquet')
df_ad.to_parquet(path+'df_ad_20210924.parquet')

In [ ]:

findcols(df_dem,'aspa')

In [37]:
df_dem['AD'].sum()

1401.0

In [11]:
df_pd['PD'].sum()

1930.0

['f00dementia_in_alzheimers_disease_f130836_0_0',
 'f01vascular_dementia_f130838_0_0',
 'f02dementia_in_other_diseases_classified_elsewhere_f130840_0_0',
 'f03unspecified_dementia_f130842_0_0']